# Obj: Create a shakespearean text generation using simple markov chain

Inspired by this youtube [video](https://www.youtube.com/watch?v=E4WcBWuQQws&ab_channel=NormalizedNerd) By Normalized Nerd

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

The corpuses are 375 poems by William Shakespeare that are scraped from this [website](https://www.williamshakespeare.net/poems.jsp)

### Scrape the links to each poem

The poems are located in a dedicated page. I will scrape the link to those page from the main page. To do that, Selenium will be needed as an interaction with the web is necessary. After obtaining the link, I will just use request and beautiful soup as they are much faster.

In [16]:
WEBSITE = "https://www.williamshakespeare.net/poems.jsp"
cService = webdriver.ChromeService("../chromedriver.exe")
driver = webdriver.Chrome(service = cService)
driver.get(WEBSITE)

# Expand the number of shown links in the main page
options = driver.find_elements(By.TAG_NAME, "option")
last_option = options[-1]  # picking the last option (100)
last_option.click()

links = []
for i in range(1, 5):  # Loop across the pages where the links are stored
    rows = driver.find_elements(By.CLASS_NAME, 'sorting_1')
    for row in rows:
        content = row.find_element(By.TAG_NAME, "a")
        link = content.get_attribute("href")
        links.append(link)
    if i < 4:  # Click the next page a long a it is not the last page
        next_ = driver.find_element(By.ID, 'poem_table_next')
        next_.click()

driver.close()

### Scrape the raw html page

In [46]:
raw_html = []
for i in links:
    req = requests.get(i)
    raw_html.append(req.text)  # obtained the links to each poem

### Cleaning and combining

In [112]:
def clean_text(text):
    # This particular function is generated by ChatGPT :p.
    # Sadly, my regex sucks hahaha
    
    roman_pattern = r'\b[IVXLCDM]+\b'

    # Use re.sub() to replace Roman numerals with an empty string
    cleaned_text = re.sub(roman_pattern, '', text)
    
    # Remove excessive whitespace and newline characters
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # extra space is fine (theres split)
    
    # Remove punctuation characters using regex pattern
    cleaned_text = cleaned_text.replace("'", "")
    cleaned_text = re.sub(r'[^\w\s]', ' ', cleaned_text)    

    return cleaned_text

def preprocessing(x):
    # converting raw html to soup object and return the cleaned text for each poem
    
    soup = BeautifulSoup(x, "html.parser")
    raw_text = soup.find("p").get_text("\n")
    cleaned_text = str.lower(clean_text(raw_text))
    return cleaned_text

### Tokenization

In [186]:
final_text = [preprocessing(i) for i in raw_html]
final_text = " ".join(final_text).split()

In [190]:
print(f"number of token: {len(final_text)}")

number of token: 54227


### The Markov Chain Model

In this model, each ngram can be considered as a state, each with connections to other ngrams

In [180]:
class MarkovChainNLP():
    
    def generate_ngram(self, text, n):
        ngrams = []
        nwords = len(text)
        for i in range(nwords - n + 1):
            gram  = tuple(text[i:i + n])
            ngrams.append(gram)
        return ngrams
    
    def get_probability(self, transition):
        # converting the connections into probability
        
        for i in transition.keys():
            sum_1 = sum(transition[i].values())  # sum of connection for each state
            for j in transition[i].keys():
                transition[i][j] = transition[i][j]/sum_1  # dividing by the sum of connection
        return transition          

    def fit(self, text, n_ngram):
        transition = {}
        ngrams = self.generate_ngram(text, n_ngram)
        for i in range(len(ngrams) - 1):
            current = ngrams[i]
            next_ = ngrams[i+1]

            if current not in transition.keys():  # add new state
                transition[current] = {}
                transition[current][next_] = 1
            else:
                if next_ in transition[current].keys():  # add the next_state's occurence
                    transition[current][next_] += 1
                else:
                    transition[current][next_] = 1  # add new next_state
        
        self.transition = self.get_probability(transition)
        
    def predict(self, prompt, limit):
        prediction = "" + prompt + " "
        X = tuple(prompt.split())
        
        for i in range(limit):
            curr_choices = self.transition[X]
            chosen_idx = np.random.choice(len(curr_choices), 
                                          p=list(curr_choices.values())) # random choice with probability
            raw_result = list(curr_choices.keys())[chosen_idx]
            prediction += raw_result[1] + " "  # only pick one word from each ngram (after the current word)
            X = raw_result
        
        return prediction
            
            

## Testing

In [181]:
model = MarkovChainNLP()
model.fit(final_text, 2)

In [191]:
print(f"number of states: {len(model.transition)}")

number of states: 22617


### Finding the prompt (initial ngram) that has the most connection

In [192]:
smallest = 1

prompt = None

second_smallest = 1
second_prompt = None

for i in model.transition:
    for j in model.transition[i]:
        if model.transition[i][j]<smallest:
            second_smallest = smallest
            second_prompt = prompt
            smallest = model.transition[i][j]
            prompt = i
            
print(f"smallest: {smallest}")
print(f"prompt: {prompt}")
print()
print(f"second_smallest: {second_smallest}")
print(f"second_prompt: {second_prompt}")

smallest: 0.00847457627118644
prompt: ('in', 'the')

second_smallest: 0.029411764705882353
second_prompt: ('in', 'their')


### Prompt: "in the"

In [194]:
for i in range(1, 21):
    print(f"{i} : {model.predict('in the', 10)}")

1 : in the breath of words respect me for thee hence o that 
2 : in the very worst of fortunes might and other strains of woe 
3 : in the midst of all too near sin of self doing crime 
4 : in the living record of your desire have no end mine appetite 
5 : in the caldron boil and bake eye of heaven better becomes the 
6 : in the bay where all thy might is more than have spent 
7 : in the bud and vaded in the dark liver of blaspheming jew 
8 : in the west which by lacking have supposèd dead and there and 
9 : in the west which by lacking have supposed dead and drowsy fire 
10 : in the breath that from him there sap checked with frost and 
11 : in the vault to whose sound chaste wings obey but thou wilt 
12 : in the suffering pangs it bears as often shrieking undistinguishd woe in 
13 : in the praise thereof spends all his face love lackd a dwelling 
14 : in the brain that ink may character which hath no exchequer now 
15 : in the world or else of thee this prognosticate thy end is 
16 : 

### Prompt: "in their"

In [195]:
for i in range(1, 21):
    print(f"{i} : {model.predict('in their', 10)}")

1 : in their glory die the earth can yield me but yet thou 
2 : in their gold coats spots you see those be rubies fairy favours 
3 : in their youthful sap at height decrease and wear their brave state 
4 : in their garments though new fangled ill some in their pride lies 
5 : in their wills obey many there were that did not better for 
6 : in their birth and where they grew was it his spirit by 
7 : in their glory die the world to say it is an ever 
8 : in their wealth some in their smells alone but in the churchway 
9 : in their youthful sap at height decrease and wear their brave state 
10 : in their garments though new fangled ill some in their glory die 
11 : in their hawks and hounds some in their birth some in their 
12 : in their hue encloses o father what a torment thrice threefold thus 
13 : in their glory die the earth can have but earth which is 
14 : in their smells alone but in my sight where wasteful time debateth 
15 : in their wealth some in their glory die the earth de

### Not bad :p

- Well, some sentences make sense, but most dont hahaha. Then again, poems are not meant to be 100% understood anyway. Let's just say the generated text are meant to be felt, not understood :p haha. 
- Perhaps it's a bad idea to use markov chain on poem as poems are very lenient in terms of grammar and vocabularies, hence a pattern becomes hard to detect. 